In [13]:
import apache_beam as beam
import glob
import os
import sys
import string

from datetime import datetime
from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.runners import DataflowRunner

import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner

In [4]:
! gsutil -m cp -r gs://text-analysis-323506/data ./
! unzip ./data/data_archive.zip 
! bzip2 -d train.ft.txt.bz2 
! bzip2 -d test.ft.txt.bz2
! mv *.ft.txt ./data/

Copying gs://text-analysis-323506/data/data_archive.zip...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Archive:  ./data/data_archive.zip]  99% Done                                    
  inflating: test.ft.txt.bz2         
  inflating: train.ft.txt.bz2        


In [15]:
def contract_line(line: str):
    contraction_dict = {"ain't": "are not", "'s":" is", "aren't": "are not", "don't": "do not", "didn't": "did not", "won't": "will not", 
                   "can't": "cannot"}
    
    words = line.split()
    for i in range(len(words)):
        if words[i] in contraction_dict:
            words[i] = contraction_dict[words[i]]
    return ' '.join(words)

In [16]:
def to_lower_case(line: str):
    return line.lower()

In [17]:
def remove_punctuations(line: str):
    return line.translate(str.maketrans('', '', string.punctuation))

In [18]:
def remove_stopwords(line: str):
    from nltk.corpus import stopwords
    
    stop_words = set(stopwords.words('english'))
    return " ".join([word for word in line.split() if word not in stop_words])

In [21]:
def remove_special_chars(line: str):
    import re
    return re.sub('[\W\_]','',line)